In [ ]:
import ee
import geemap as gm
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [ ]:
import skiba.buffer_method as bm

In [ ]:
m = bm.buffer_method().vbox
m

In [ ]:
def create_buffers(data, radius_meters, output_shp):
    if isinstance(data, str):
        coordinates = pd.read_csv(data)
        gdf = gpd.GeoDataFrame(
            coordinates,
            geometry=gpd.points_from_xy(coordinates.LON, coordinates.LAT),
            crs="EPSG:4326",  # Directly set CRS during creation
        )
    elif isinstance(data, pd.DataFrame):
        coordinates = data
        gdf = gpd.GeoDataFrame(
            coordinates,
            geometry=gpd.points_from_xy(coordinates.LON, coordinates.LAT),
            crs="EPSG:4326",  # Directly set CRS during creation
        )
    else:
        gdf = data.to_crs(epsg=4326)  # Ensure WGS84

    # Convert to UTM for accurate buffering
    utm_crs = gdf.estimate_utm_crs()
    buffered = gdf.to_crs(utm_crs).buffer(radius_meters).to_crs("EPSG:4326")

    # Save buffers as shapefile
    buffer_gdf = gpd.GeoDataFrame(geometry=buffered)
    buffer_gdf.to_file(output_shp)
    return buffer_gdf


file_path = "../data/selected_coordinates.csv"
buffer = 1000
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
output_file = f"{buffer}.shp"
out_path = os.path.join(out_dir, output_file)
shapefile = create_buffers(file_path, buffer, out_path)


# 2. Extract median values from GEE
def extract_median_values(buffer_shp, output_csv, scale=500):
    # Load buffers
    fc = gm.shp_to_ee(buffer_shp)

    # Example dataset (replace with your own)
    geeimage = buffer_method.load_gee_as_image(geedata, start_date, end_date)

    name = geedata
    file_name = name.replace("/", "_")

    out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
    output_file = f"{file_name}.csv"
    out_path = os.path.join(out_dir, output_file)

    # Extract values
    gm.extract_values_to_points(fc, median_img, output_csv, scale=scale)

In [ ]:
import skiba.map as map

map = map.Map()
map
shapefile_path = "../data/1000.shp"
map.add_shp(shapefile_path)